# 1. Очистка и подготовка данных

### Импорт модулей и загрузка функций

In [1]:
# load modules and functions
import os
import re                                 
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import time, timedelta
from collections import Counter
from statistics import median
from statistics import mode

from typing import NoReturn               
from pandas import DataFrame              
from colorama import Fore, Back, Style    
from tqdm import tqdm # progress bar                    

import My_Function_050824_M_Filimonov as mvf # loading functions written by me for the Project from a file 

import importlib
importlib.reload(mvf)  # Перезагружаем модуль после изменений

<module 'My_Function_050824_M_Filimonov' from 'C:\\Users\\fmikh\\Documents\\ICH\\FIN_PRJ\\My_Function_050824_M_Filimonov.py'>

## 1.1 CALLS

### 1.1.1 Загрузка данных и преобразование в датафрейм

In [2]:
# Forming a path with 'os.path'
file_path = os.path.join("Calls (Done).xlsx")

try:
    print("Loading 'Calls' Dataframe...")
    # Timing the loading process
    %time calls = calls = pd.read_excel(file_path, dtype={"Id": str, "CONTACTID": str})
    calls.name = 'calls'
    print("Dataframe successfully loaded!")
    display(calls.head(3))
except FileNotFoundError:
    print("Error: The file 'calls.xlsx' was not found in the specified path.")
except ValueError:
    print("Error: The file 'calls.xlsx' could not be read as an Excel file. Ensure the file format is valid.")
except Exception as e:
    print(f"An unexpected error occurred during file loading: {e}")

Loading 'Calls' Dataframe...
CPU times: total: 16.5 s
Wall time: 18.5 s
Dataframe successfully loaded!


,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Dialled Number,Outgoing Call Status,Scheduled in CRM,Tag
0,5805028000000805001,30.06.2023 08:43,John Doe,NaN,Inbound,171.0,Received,NaN,NaN,NaN,NaN
1,5805028000000768006,30.06.2023 08:46,John Doe,NaN,Outbound,28.0,Attended Dialled,NaN,Completed,0.0,NaN
2,5805028000000764027,30.06.2023 08:59,John Doe,NaN,Outbound,24.0,Attended Dialled,NaN,Completed,0.0,NaN


***Информации о типах данных и пропущенных значениях***

In [3]:
# output information about data types and missing values
#calls.describe(include='all').T
mvf.my_df_info(calls,'calls')

'calls' DataFrame size (row, column): (95874, 11)
DataTypes info:
Id                             object
Call Start Time                object
Call Owner Name                object
CONTACTID                      object
Call Type                      object
Call Duration (in seconds)    float64
Call Status                    object
Dialled Number                float64
Outgoing Call Status           object
Scheduled in CRM              float64
Tag                           float64
dtype: object
Number of missing values:
Id                                0
Call Start Time                   0
Call Owner Name                   0
CONTACTID                      3933
Call Type                         0
Call Duration (in seconds)       83
Call Status                       0
Dialled Number                95874
Outgoing Call Status           8999
Scheduled in CRM               8999
Tag                           95874
dtype: int64


### 1.1.2 Предварительная очистка: 
- удаляются дубликаты строк
- удаляются пустые строки и столбцы
- пробелы в названиях столбцов заменяются символом подчеркивания 

In [4]:
# DataFrame pre-cleaning
calls.name = 'calls'
mvf.df_pre_clean(calls)

# Rename column Call_Duration_(in_seconds)'-> 'Call_Duration'
calls.rename(columns={'Call_Duration_(in_seconds)': 'Call_Duration'}, inplace=True)

DataFrame 'calls' info:

ROWs number: 95874
COLUMNs number: 11
No ROWs, where all values are NaN found
No duplicated ROWs found.
COLUMNs with all NaN values removed:
- Tag
- Dialled Number

Old columns names: ['Id', 'Call Start Time', 'Call Owner Name', 'CONTACTID', 'Call Type', 'Call Duration (in seconds)', 'Call Status', 'Outgoing Call Status', 'Scheduled in CRM']

New columns names: ['Id', 'Call_Start_Time', 'Call_Owner_Name', 'CONTACTID', 'Call_Type', 'Call_Duration_(in_seconds)', 'Call_Status', 'Outgoing_Call_Status', 'Scheduled_in_CRM']



#### Вывод по предварительной очистке датафрейма Calls:
- удалено два столбца *(Dialled Number, Tag)* как неинформативные. 
- дубликатов сторок не выявлено

### 1.1.3 Обработка отсутствующих значений и преобразование типов данных

#### Анализ данных по колонкам

In [5]:
# Analyzing columns 
mvf.my_columns_describe(calls)


DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id                    95874 non-null  object 
 1   Call_Start_Time       95874 non-null  object 
 2   Call_Owner_Name       95874 non-null  object 
 3   CONTACTID             91941 non-null  object 
 4   Call_Type             95874 non-null  object 
 5   Call_Duration         95791 non-null  float64
 6   Call_Status           95874 non-null  object 
 7   Outgoing_Call_Status  86875 non-null  object 
 8   Scheduled_in_CRM      86875 non-null  float64
dtypes: float64(2), object(7)
memory usage: 6.6+ MB
None

DataFrame describe:
Id                         0
Call_Start_Time            0
Call_Owner_Name            0
CONTACTID               3933
Call_Type                  0
Call_Duration             83
Call_Status                0
Outgoing_Call_Status    8999
Sche

####  Предложения по преобразованию типов в датафрейме Calls:

In [6]:
#  #   Column                  Dtype     new Dtype   
# ---  ------                  -----      ------
#  0   Id                      object  -> object   - тип данных без изменений, идентификатор
#  1   Call_Start_Time         object  -> datetime - для анализа временных рядов
#  2   Call_Owner_Name         object  -> category - имена повторяются и их набор фиксирован *
#  3   CONTACTID               object  -> object   - тип данных без изменений, идентификатор
#  4   Call_Type               object  -> category - значения повторяются и ти набор фиксирован *
#  5   Call_Duration           float64 -> int32    - т.к. это целые числа
#  6   Call_Status             object  -> category - значения повторяются и их набор фиксирован *
#  7   Outgoing_Call_Status    object  -> category - значения повторяются и их набор фиксирован *
#  8   Scheduled_in_CRM        float64 -> category - значения повторяются и их набор фиксирован 
#                                                       [1.0,0.0,NaN]->["yes","no","unknown"]*

# * ускорит операции фильтрации и группировки

#### Предложения по замене пропущенных значений в датафрейме Calls:

- **CONTACTID** т.к это идентификатор, пропущенные значения (4%) ***целесообразно заполнить модой***, из групп по Call_Start_Time (предварительно выделив из него дату) ,Call_Owner_Name и Call_Type для сохранения целостности данных и корректного объединения датафреймов (таблиц) при дальнейшем анализе.

- **Call_Duration** пропущенные значения целесообразно ***заполниить медианным значением*** (***8 сек***) вместо среднего по столбцу (***165 сек***). Это позволит  нивелировать влияние "выбросов" (очень длинных звонков) и сохранить тенденцию *типичной длительности звонков*.

- **Outgoing_Call_Status** пропущенные значения целесообразно заполнить значением ***"unknown"***, которое обозначит, что статус исходящего звонка не был указан. Это позволит сохранить структуру данных для последующего анализа.


- **Scheduled in CRM** пропущенные значения целесообразно заполнить значением ***"unknown"***, которое указывает на то, что нам неизвестно, был ли звонок запланирован в CRM.

#### Выполняем преобразование типов и обработку отсутствующих значений 

In [7]:
# Convert Call_Start_Time to datetime format
calls['Call_Start_Time'] = pd.to_datetime(calls['Call_Start_Time'], 
                                          format='%d.%m.%Y %H:%M', 
                                          dayfirst=True, 
                                          errors='coerce')

# add a temporate column with date for grouping
calls['Call_Start_Date'] = calls['Call_Start_Time'].dt.date  # extract date from Call_Start_Time

# function to fill empty CONTACTID values with column group mode ['Call_Start_Date', 'Call_Owner_Name', 'Call_Type']
def fill_with_mode(group):
    if not group['CONTACTID'].mode().empty:
        mode_value = group['CONTACTID'].mode()[0]  
        group['CONTACTID'] = group['CONTACTID'].fillna(mode_value)  # Fill in the blanks by creating a new copy
    return group

# Tracking the number of missing values before and after processing
# Number of missing values in 'CONTACTID' before processing
num_NaN_before = calls['CONTACTID'].isna().sum()

# Applying 'fill_adgroup' to fill missing AdGroup values
calls = calls.groupby(['Call_Start_Date', 'Call_Owner_Name', 'Call_Type'], observed=True)\
             .apply(fill_with_mode,include_groups=False)\
             .reset_index(drop=False)  

# Number of missing values in 'CONTACTID' after processing
num_NaN_after = calls['CONTACTID'].isna().sum()

# Calculate the number of processed rows and the percentage of missing values processed
processed_rows = num_NaN_before - num_NaN_after
percent_processed = (processed_rows / num_NaN_before) * 100  # Percentage of processed NaN values

# Output the results
print(Fore.BLUE + f"Number of NaN records in 'CONTACTID' before processing = {num_NaN_before}")
print(f"Number of NaN records in 'CONTACTID' after processing = {num_NaN_after}")
print(f"Processed NaN are {processed_rows} values, which is {percent_processed:.2f}%." + Style.RESET_ALL)

# remove temporate columns
calls = calls.drop(columns=['Call_Start_Date','level_3'])

# delete records where CONTACTID is NaN
calls = calls.dropna(subset=['CONTACTID'])

# calls.Call_Duration is filled with the median and converted to int32
calls.Call_Duration = calls.Call_Duration.fillna(calls['Call_Duration'].median())
calls.Call_Duration = calls.Call_Duration.astype('int32')

# list of columns to convert to category data type
category_columns = ['Call_Owner_Name', 'Call_Type', 'Call_Status', 'Outgoing_Call_Status', 'Scheduled_in_CRM']
for col in category_columns:
    calls[col] = calls[col].astype('category')

# fill in missing values 'unknown'
for col in ['Outgoing_Call_Status', 'Scheduled_in_CRM']:
    if 'unknown' not in calls[col].cat.categories:
        calls[col] = calls[col].cat.add_categories(['unknown'])
    calls[col] = calls[col].fillna('unknown')

# Scheduled_in_CRM values "1.0" are replaced with "yes" and "0.0" with "no" -- for convenience
calls['Scheduled_in_CRM'] = calls['Scheduled_in_CRM'].cat.rename_categories({1.0: 'yes', 0.0: 'no'})

mvf.my_df_info(calls,'calls')

Number of NaN records in 'CONTACTID' before processing = 3933
Number of NaN records in 'CONTACTID' after processing = 286
Processed NaN are 3647 values, which is 92.73%.
'calls' DataFrame size (row, column): (95588, 9)
DataTypes info:
Call_Owner_Name               category
Call_Type                     category
Id                              object
Call_Start_Time         datetime64[ns]
CONTACTID                       object
Call_Duration                    int32
Call_Status                   category
Outgoing_Call_Status          category
Scheduled_in_CRM              category
dtype: object
Number of missing values:
Call_Owner_Name         0
Call_Type               0
Id                      0
Call_Start_Time         0
CONTACTID               0
Call_Duration           0
Call_Status             0
Outgoing_Call_Status    0
Scheduled_in_CRM        0
dtype: int64


#### Общие выводы по датафрейму Calls:
1. Изначально датафрейм имел `95874 строк и 11 колонок`.
2. После предварительной очистки было удалено два пустых столбца (Dialled Number, Tag) как неинформативные, дубликатов и пустых сторок не выявлено.
3. На основании анализа типов данных и их гомогенности, а также распределения отсутствующих значений приняты решения по изменению типов и категорированию данных. Также для каждого столбца выбрана своя тактика работы с отсутствующими значениями.
4. После очистки данных датафрейм имеет `95588 строк и 9 столбцов`.

### 1.1.4 Сохранение CALLS

In [8]:
# write the cleaned dataframe to an Excel file to build a dashboard in Power BI
calls.to_excel('calls_cleaned.xlsx', index=False, engine='openpyxl')

#запись датафрейма в файл
calls.to_pickle("01_calls.pkl")

## 1.2 CONTACTS

### 1.2.1 Загрузка данных и преобразование в датафрейм

In [9]:
# forming a pass with 'os.path'
file_path = os.path.join("Contacts (Done).xlsx")

# Reading Excel file
contacts = pd.read_excel(file_path, dtype={"Id": str})
contacts.name = 'contacts'
display(contacts.head())

mvf.my_df_info(contacts,'contacts')

,Id,Contact Owner Name,Created Time,Modified Time
0,5805028000000645014,Rachel White,27.06.2023 11:28,22.12.2023 13:34
1,5805028000000872003,Charlie Davis,03.07.2023 11:31,21.05.2024 10:23
2,5805028000000889001,Bob Brown,02.07.2023 22:37,21.12.2023 13:17
3,5805028000000907006,Bob Brown,03.07.2023 05:44,29.12.2023 15:20
4,5805028000000939010,Nina Scott,04.07.2023 10:11,16.04.2024 16:14


'contacts' DataFrame size (row, column): (18548, 4)
DataTypes info:
Id                    object
Contact Owner Name    object
Created Time          object
Modified Time         object
dtype: object
Number of missing values:
Id                    0
Contact Owner Name    0
Created Time          0
Modified Time         0
dtype: int64


### 1.2.2 Предварительная очистка:

In [10]:
# DataFrame pre-cleaning
mvf.df_pre_clean(contacts)

DataFrame 'contacts' info:

ROWs number: 18548
COLUMNs number: 4
No ROWs, where all values are NaN found
No duplicated ROWs found.
No COLUMNs with all NaN values were found.


Old columns names: ['Id', 'Contact Owner Name', 'Created Time', 'Modified Time']

New columns names: ['Id', 'Contact_Owner_Name', 'Created_Time', 'Modified_Time']



#### Вывод по предварительной очистке датафрейма Сontacts:
- путые строки и колонки отсутствуют
- дублирующиеся строки отсутствуют

### 1.2.3 Обработка отсутствующих значений и преобразование типов данных   

#### Анализ данных по колонкам

In [11]:
# Analyzing columns 
mvf.my_columns_describe(contacts)


DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id                  18548 non-null  object
 1   Contact_Owner_Name  18548 non-null  object
 2   Created_Time        18548 non-null  object
 3   Modified_Time       18548 non-null  object
dtypes: object(4)
memory usage: 579.8+ KB
None

DataFrame describe:
Id                    0
Contact_Owner_Name    0
Created_Time          0
Modified_Time         0
dtype: int64

List of OBJECT-type columns
Column: 'Id'
Unique values: 18548
Missing values (NaN): 0 | 0%

Column: 'Contact_Owner_Name'
Attention! The data in the column is not homogeneous:
Contact_Owner_Name
<class 'str'>     18547
<class 'bool'>        1
Name: count, dtype: int64
Unique values: 28
Values distribution:
Contact_Owner_Name
Charlie Davis      2018
Ulysses Adams      1816
Julia Nelson       1769
Paula Under

#### Исправление ошибки гомогенности данных в столбце Contact_Owner_Name

In [12]:
# Convert Created_Time to datetime format
contacts['Created_Time'] = pd.to_datetime(contacts['Created_Time'], 
                                          format='%d.%m.%Y %H:%M', 
                                          dayfirst=True, 
                                          errors='coerce')

# identify the erroneous value in the 'Contact_Owner_Name' column 
# looking for a row in the Contact_Owner_Name column with the value False
false_row = contacts[contacts['Contact_Owner_Name'] == False]
print(Fore.RED +"Row with incorrect Contact_Owner_Name:\n"+ Style.RESET_ALL)
display(false_row)

# replace folse value with the Contact_Owner_Name mode from the group by creation date

if not false_row.empty:
    # take the Created_Time value for the found row
    ct = false_row.iloc[0]['Created_Time']
    
    # Group the DataFrame by Created_Time and calculate the mode for Contact_Owner_Name
    contacts['Created_Date'] = contacts['Created_Time'].dt.date
    contacts_group = contacts[contacts['Created_Date'] == ct.date()]
    
    # Determine the most common Contact_Owner_Name
    most_common_name = contacts_group['Contact_Owner_Name'].mode()
    
    if not most_common_name.empty:
        # Update the Contact_Owner_Name value in the found row
        contacts.loc[contacts['Contact_Owner_Name'] == False, 'Contact_Owner_Name'] = most_common_name[0]
        print(Back.RED + f"Значение Contact_Owner_Name в ошибочной строке заменено на: {most_common_name[0]}"+ Style.RESET_ALL)
    else:
        print(Fore.RED + "Unable to determine the most common Contact_Owner_Name."+ Style.RESET_ALL)
else:
    print(Back.WHITE + "A row with a value of False in the Contact_Owner_Name column was not found."+ Style.RESET_ALL)
    
# deletind temporated column Created_Date
contacts = contacts.drop(columns=['Created_Date'])

Row with incorrect Contact_Owner_Name:



,Id,Contact_Owner_Name,Created_Time,Modified_Time
2197,5805028000008772190,False,2023-09-24 09:01:00,13.10.2023 16:44


Значение Contact_Owner_Name в ошибочной строке заменено на: Jane Smith


#### Предложения по преобразованию типов в датафрейме Contacts:

In [13]:
# #   Column                Dtype      New Dtype
# ---  ------               -----      ---------
#  0   Id                   object   ->    object      - оставить без изменений  
#  1   Contact_Owner_Name   object   ->   category - значения повторяются и их набор фиксирован * 
#  2   Created_Time         object   ->   datatime    - для анализа временных рядов
#  3   Modified_Time        object   ->   datatime    - для анализа временных рядов
#  * ускорит операции фильтрации и группировки

#### Выполняем преобразование типов и обработку отсутствующих значений 

In [14]:
contacts['Contact_Owner_Name'] = contacts['Contact_Owner_Name'].astype('category')

contacts['Modified_Time'] = pd.to_datetime(contacts['Modified_Time'], format='%d.%m.%Y %H:%M', dayfirst=True, errors='coerce')

mvf.my_df_info(contacts,'contacts')

'contacts' DataFrame size (row, column): (18548, 4)
DataTypes info:
Id                            object
Contact_Owner_Name          category
Created_Time          datetime64[ns]
Modified_Time         datetime64[ns]
dtype: object
Number of missing values:
Id                    0
Contact_Owner_Name    0
Created_Time          0
Modified_Time         0
dtype: int64


#### Общие выводы по датафрейму Contacts:
- исправили ошибку гомогенности данных Contact_Owner_Name; 
- пропущенных значений нет;
- типы данных преобразованы в соотвествии с выводами из анализа датасета

### 1.2.4 Сохранение CONTACTS

In [15]:
# write the cleaned dataframe to an Excel file to build a dashboard in Power BI
contacts.to_excel('contacts_cleaned.xlsx', index=False, engine='openpyxl')

#запись датафрейма в файл
contacts.to_pickle("02_contacts.pkl")

## 1.3 SPEND

### 1.3.1 Загрузка данных и преобразование в датафрейм

In [16]:
# forming a pass with 'os.path'
file_path = os.path.join("Spend (Done).xlsx")

# Reading Excel file
spend = pd.read_excel(file_path)
spend.name = 'spend'
display(spend.head())

mvf.my_df_info(spend,'spend')

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
0,2023-07-03,Google Ads,gen_analyst_DE,6,0.00,0,NaN,NaN
1,2023-07-03,Google Ads,performancemax_eng_DE,4,0.01,1,NaN,NaN
2,2023-07-03,Facebook Ads,NaN,0,0.00,0,NaN,NaN
3,2023-07-03,Google Ads,NaN,0,0.00,0,NaN,NaN
4,2023-07-03,CRM,NaN,0,0.00,0,NaN,NaN


'spend' DataFrame size (row, column): (20779, 8)
DataTypes info:
Date           datetime64[ns]
Source                 object
Campaign               object
Impressions             int64
Spend                 float64
Clicks                  int64
AdGroup                object
Ad                     object
dtype: object
Number of missing values:
Date              0
Source            0
Campaign       5994
Impressions       0
Spend             0
Clicks            0
AdGroup        6828
Ad             6828
dtype: int64


### 1.3.2 Предварительная очистка:

In [17]:
# DataFrame pre-cleaning
mvf.df_pre_clean(spend)

DataFrame 'spend' info:

ROWs number: 20779
COLUMNs number: 8
No ROWs, where all values are NaN found
Number of duplicate rows: 917
All duplicates ROWs have been removed.
No COLUMNs with all NaN values were found.


Old columns names: ['Date', 'Source', 'Campaign', 'Impressions', 'Spend', 'Clicks', 'AdGroup', 'Ad']

New columns names: ['Date', 'Source', 'Campaign', 'Impressions', 'Spend', 'Clicks', 'AdGroup', 'Ad']



#### Вывод по предварительной очистке датафрейма Spend:
- путые строки и колонки отсутствуют
- выявлено 917 дубликатов строк. Дубликаты удалены.

### 1.3.3 Обработка отсутствующих значений и преобразование типов данных   

#### Анализ данных по колонкам

In [18]:
# Analyzing columns 
mvf.my_columns_describe(spend)


DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 19862 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         19862 non-null  datetime64[ns]
 1   Source       19862 non-null  object        
 2   Campaign     14785 non-null  object        
 3   Impressions  19862 non-null  int64         
 4   Spend        19862 non-null  float64       
 5   Clicks       19862 non-null  int64         
 6   AdGroup      13951 non-null  object        
 7   Ad           13951 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 1.4+ MB
None

DataFrame describe:
Date              0
Source            0
Campaign       5077
Impressions       0
Spend             0
Clicks            0
AdGroup        5911
Ad             5911
dtype: int64

List of OBJECT-type columns
Column: 'Source'
Unique values: 14
Values distribution:
Source
Facebook Ads      

####  Предложения по преобразованию типов в датафрейме Spend:
- **Date**           datetime64[ns] -> оставить без изменений
- **Source**                object -> преобразовать в категориальный тип данных
- **Campaign**               object -> преобразовать в категориальный тип данных
- **Impressions**             int64 -> преобразовать в Int32
- **Spend**                float64 -> оставить без изменений
- **Clicks**                  int64 -> преобразовать в Int32
- **AdGroup**                object -> преобразовать в категориальный тип данных
- **Ad**                     object -> преобразовать в категориальный тип данных


#### Предложения по замене пропущенных значений в датафрейме Spend:
- **Campaign** - пропущенные значения возможно частично заменить на основе подходящего значения из группы по Source (по совпадению с датой указанной в названии Campaign), остальные пропуски заполним значением "unknown". 
- **AdGroup** - пропущенные значения возможно частично заменить на основе столбца «Campaign» с использованием предопределенной логики, остальные пропуски заполним значением "unknown".  
- **Ad** - пропуски заполним значением "unknown".

In [19]:
# # group by Source and view unique Campaign values
# grouped_campaigns = spend.groupby('Source')['Campaign'].apply(lambda x: x.dropna().unique())

# for source, campaigns in grouped_campaigns.items():
#     print(f"Group name: {source}")
#     print(f"unique values count: {len(campaigns)}")
#     print(f"List of unique values: {list(campaigns)}")
#     print("-" * 40)

#### Выполняем преобразование типов и обработку отсутствующих значений 

In [20]:
'''
Initial dictionary (Python code is given in the cell above): 
 - keys - 'Source' unique values;
 - values - List of unique values 'Campaign' groupped by 'Source'
 - groups with number of values > 0 were selected.
'''

unique_campaigns_by_source = {
    'Facebook Ads': ['03.07.23women', '02.07.23wide_DE', '05.07.23interests_DE', '04.07.23recentlymoved_DE',
                     '07.07.23LAL_DE', '10.07.23wide_com_DE', '15.07.23b_DE', '02.08.23interests_DE',
                     '12.09.23interests_Uxui_DE', '24.09.23retargeting_DE', '18.10.23wide_gos_DE', '30.11.23wide_DE',
                     '07.12.23test_DE', '09.02.24berlin_dd_DE', '17.03.24wide_AT', '15.03.24recentlymoved_AT',
                     '20.03.24_widde_PL', '20.03.24interests_WebDev_AT', '20.03.24interests_WebDev_PL',
                     '20.03.24recentlymoved_PL', '01.04.23women_PL', '15.04.24LAL_ab__PL', '20.05.24interests_DE',
                     '20.05.24wide_DE', '12.06.24wide_DE'],
    'Google Ads': ['gen_analyst_DE', 'performancemax_eng_DE', 'comp_search_DE', 'brand_search_eng_DE', 'discovery_DE',
                   'discovery_wide1_AT', 'performancemax_wide_AT', '1performancemax_wide_PL'],
    'Test': ['07.12.23test_DE', '02.05.24test_DE'],
    'Tiktok Ads': ['12.07.2023wide_DE', '24.07.2023wide_DE', '05.09.2023wide_DE', '15.03.2024wide_AT',
                   '20.03.2024wide_PL', '22.05.2024wide_DE'],
    'Webinar': ['web2408_DE', '14.11.23wide_webinar_DE', '15.11.23wide_webinar_DE', 'blog2_DE',
                '01.02.24wide_webinar_DE', 'bbo_DE', '08.04.24wide_webinar_DE', '08.06.24wide_webinar_DE'],
    'Youtube Ads': ['youtube_shorts_DE', 'youtube_shortsin_AT', 'shorts_PL'],
}

# Function to extract a date from a string
def extract_date_from_string(text: str) -> pd.Timestamp:
    """
    Extracts a date from a given text string.

    Parameters:
    text (str): The input string that may contain a date.

    Returns:
    pd.Timestamp: A datetime object if a valid date is found, otherwise None.

    Details:
    - Searches for a date in the formats 'dd.mm.yyyy' or 'dd.mm.yy' using a regular expression.
    - Tries to parse the extracted date into a Pandas Timestamp object.
    - Returns None if no valid date is found or if parsing fails.
    """
    match = re.search(r'\d{2}\.\d{2}\.\d{4}|\d{2}\.\d{2}\.\d{2}', text)
    if match:
        date_str = match.group()
        try:
            return pd.to_datetime(date_str, format='%d.%m.%Y')
        except ValueError:
            try:
                return pd.to_datetime(date_str, format='%d.%m.%y')
            except ValueError:
                return None
    return None


# Function to replace missing Campaign values with values based on Source
def fill_campaign(row: pd.Series) -> str:
    """
    Fills missing 'Campaign' values based on the 'Source' and the campaign dictionary.

    Parameters:
    row (pd.Series): A row from the DataFrame.

    Returns:
    str: The filled Campaign value or the original value if not missing.

    Details:
    - Checks if the 'Campaign' value is missing (NaN).
    - Looks up possible campaign values based on the 'Source' from the predefined dictionary.
    - If a campaign date matches the current row's date, assigns it to the row.
    - If no date matches, assigns the first campaign from the list in the dictionary.
    - Returns the existing value if 'Campaign' is not missing.
    """
    if pd.isnull(row['Campaign']):
        source = row['Source']
        current_date = row['Date']
        if source in unique_campaigns_by_source:
            campaigns = unique_campaigns_by_source[source]
            for campaign in campaigns:
                campaign_date = extract_date_from_string(campaign)
                if campaign_date == current_date:
                    return campaign
            if campaigns:
                return campaigns[0]
    return row['Campaign']


# Number of missing values before processing
num_NaN_before = spend['Campaign'].isna().sum()

# Applying fill_campaign()
spend['Campaign'] = spend.apply(fill_campaign, axis=1)

# Number of missing values after processing
num_NaN_after = spend['Campaign'].isna().sum()
processed_rows = num_NaN_before - num_NaN_after
percent_processed = (processed_rows / num_NaN_before) * 100

# Result output
print(Fore.BLUE + f"Number NaN records in 'Campaign' before processing = {num_NaN_before}")
print(f"Number NaN records in 'Campaign' after processing = {num_NaN_after}")
print(f"Processed NaN are {processed_rows} values, which is {percent_processed:.2f}%" + Style.RESET_ALL)

Number NaN records in 'Campaign' before processing = 5077
Number NaN records in 'Campaign' after processing = 3230
Processed NaN are 1847 values, which is 36.38%


In [21]:
# Function to fill missing AdGroup values based on Campaign (values analyzed using LMM)
# Input data: unique values lists of Campaign and AdGroup columns

def fill_adgroup(row: pd.Series) -> str:
    """
    Fills missing 'AdGroup' values based on 'Campaign' column using predefined logic.

    Parameters:
    row (pd.Series): A row of the DataFrame containing 'Campaign' and 'AdGroup' columns.

    Returns:
    str: The appropriate 'AdGroup' value based on keywords in 'Campaign', or the existing value if 'AdGroup' is not missing.

    Details:
    - Checks if the 'AdGroup' value is missing (NaN).
    - Converts the 'Campaign' value to lowercase for case-insensitive processing.
    - Searches for specific keywords in 'Campaign' and assigns corresponding 'AdGroup' values:
        - Keywords like 'wide', 'recentlymoved', 'women', etc., are mapped to respective 'AdGroup' categories.
        - Additional nested logic is applied for keywords such as 'com' or 'interests'.
    - If no keywords match, assigns a default value ('b').
    """
    if pd.isnull(row['AdGroup']) and isinstance(row['Campaign'], str):  # Check if AdGroup is missing and Campaign is a string
        campaign = row['Campaign'].lower()  # Convert to lowercase for consistent matching
        # Mapping AdGroup values based on keywords
        if 'wide' in campaign:
            return 'wide'
        elif 'recentlymoved' in campaign:
            return 'recentlymoved'
        elif 'women' in campaign:
            return 'women'
        elif 'interests' in campaign:
            if 'webdev' in campaign:
                return 'interest_work_WebDev'
            elif 'programming' in campaign:
                return 'interest_programming_WebDev'
            elif 'dataanalytics' in campaign:
                return 'interest_dataanalytics'
            else:
                return 'interest_all'
        elif 'lal' in campaign:
            return 'LAL1'
        elif 'retargeting' in campaign:
            return 'retargeting'
        elif 'discovery' in campaign:
            return 'discovery'
        elif 'python-developer' in campaign:
            return 'interest_python-developer'
        elif 'qa-engineer' in campaign:
            return 'wide_qa-engineer'
        elif 'webdesigner' in campaign:
            return 'wide_webdesigner'
        elif 'accountant' in campaign:
            return 'accountant_wide'
        elif 'berlin' in campaign:
            return 'berlin_wide'
        elif 'com' in campaign:
            if 'august' in campaign:
                return 'Com_august'
            elif 'march' in campaign:
                return 'Com_march'
            elif 'july' in campaign:
                return 'Com_july_1'
        elif 'promoposts' in campaign:
            return 'promoposts_b'
        else:
            return 'b'  # Default value for unmatched cases
    return row['AdGroup']  # Return existing value if AdGroup is not missing


# Tracking the number of missing values before and after processing
# Number of missing values in 'AdGroup' before processing
num_NaN_before = spend['AdGroup'].isna().sum()

# Applying 'fill_adgroup' to fill missing AdGroup values
spend['AdGroup'] = spend.apply(fill_adgroup, axis=1)

# Number of missing values in 'AdGroup' after processing
num_NaN_after = spend['AdGroup'].isna().sum()

# Calculate the number of processed rows and the percentage of missing values processed
processed_rows = num_NaN_before - num_NaN_after
percent_processed = (processed_rows / num_NaN_before) * 100  # Percentage of processed NaN values

# Output the results
print(Fore.GREEN + f"Number of NaN records in 'AdGroup' before processing = {num_NaN_before}")
print(f"Number of NaN records in 'AdGroup' after processing = {num_NaN_after}")
print(f"Processed NaN are {processed_rows} values, which is {percent_processed:.2f}%." + Style.RESET_ALL)

Number of NaN records in 'AdGroup' before processing = 5911
Number of NaN records in 'AdGroup' after processing = 3167
Processed NaN are 2744 values, which is 46.42%.


In [22]:
spend.dtypes

Date           datetime64[ns]
Source                 object
Campaign               object
Impressions             int64
Spend                 float64
Clicks                  int64
AdGroup                object
Ad                     object
dtype: object

In [23]:
# Source: object -> category
spend['Source'] = spend['Source'].astype('category')

# Campaign: object -> category
spend['Campaign'] = spend['Campaign'].astype('category')

# AdGroup: object -> category
spend['AdGroup'] = spend['AdGroup'].astype('category')

# Ad: object -> category
spend['Ad'] = spend['Ad'].astype('category')

# Impressions: int64 -> Int32
spend['Impressions'] = spend['Impressions'].astype('Int32')

# Clicks: int64 -> Int32
spend['Clicks'] = spend['Clicks'].astype('Int32')

# List for filling missing values of category data on "unknoun"
category_columns = ['Source','Campaign','AdGroup','Ad']

for col in category_columns:
    # Check for the 'unknown' category before adding
    if 'unknown' not in spend[col].cat.categories:
        spend[col] = spend[col].cat.add_categories(['unknown'])
    spend[col] = spend[col].fillna('unknown')
    
spend.dtypes                                  

Date           datetime64[ns]
Source               category
Campaign             category
Impressions             Int32
Spend                 float64
Clicks                  Int32
AdGroup              category
Ad                   category
dtype: object

#### Для дальнейшего анализа
в датафрейм добавим столбец AdvCountry, который  определяет страну, для целевой аудитории которой показывается реклама реклама. Столбец формируется на основе  последних двух символов значений колонки «Campaign».

In [24]:
'''
For further analysis, we will add the AdvCountry column,
which defines the country for whose target audience the 
advertisement is shown. The column is formed based on the
last two characters of the Campaign column values.
'''
def determine_country(campaign: str) -> str:
    """
    Determines the country to which the advertisement is directed 
    based on the last two characters of the 'Campaign' string.

    Parameters:
    campaign (str): A string from the 'Campaign' column that may contain location identifiers.

    Returns:
    str: The country name ('Germany', 'Poland', 'Austria', or 'other') based on the suffix.

    Logic:
    - Checks if 'Campaign' is NaN (missing value). If true, returns 'other'.
    - Extracts the last two characters of the string using slicing.
    - Matches suffix:
        - 'DE' -> 'Germany'
        - 'PL' -> 'Poland'
        - 'AT' -> 'Austria'
        - Any other suffix -> 'other'
    """
    if pd.isnull(campaign):  # Check for missing value
        return 'other'
    suffix = campaign[-2:]  # Extract last two characters
    if suffix == 'DE':       # Germany
        return 'Germany'
    elif suffix == 'PL':     # Poland
        return 'Poland'
    elif suffix == 'AT':     # Austria
        return 'Austria'
    else:                    # Other cases
        return 'other'

# Create a new column 
spend['AdvCountry'] = spend['Campaign'].apply(determine_country)

In [25]:
spend.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19862 entries, 0 to 20778
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         19862 non-null  datetime64[ns]
 1   Source       19862 non-null  category      
 2   Campaign     19862 non-null  category      
 3   Impressions  19862 non-null  Int32         
 4   Spend        19862 non-null  float64       
 5   Clicks       19862 non-null  Int32         
 6   AdGroup      19862 non-null  category      
 7   Ad           19862 non-null  category      
 8   AdvCountry   19862 non-null  object        
dtypes: Int32(2), category(4), datetime64[ns](1), float64(1), object(1)
memory usage: 921.5+ KB


#### Общие выводы по датафрейму SPEND

- Всего было 20779 строк и 8 столбцов;
- Удалено 917 дублирующихся строк;
- Проведение сопоставительного анализа в группах данных Source-Campaign и Campaign-AdGroup позволило восстановить 36.38% отсутствующих значений в Campaign и 46.42% значений в AdGroup;
- Типы данных преобразованы в соотвествии с выводами из анализа датасета

### 1.3.4 Сохранение SPEND

In [26]:
# delete not informative columns
spend = spend.drop(columns=['Ad', 'AdvCountry'])

# write the resulting dataframe to an Excel file for further processing in Power BI
spend.to_excel('spend_cleaned.xlsx', index=False, engine='openpyxl')

#запись датафрейма в файл
spend.to_pickle("03_spend.pkl")

## 1.4 Table DEALS

In [27]:
# forming a pass with 'os.path'
file_path = os.path.join("Deals (Done).xlsx")

# Reading Excel file
#skipfooter=2 - removes the last 2 lines, which are empty (the last line of the Education Type column contains an error value "#REF!")
deals = pd.read_excel(
    file_path,
    parse_dates=['Closing Date', 'Created Time'],
    date_format='%d.%m.%Y %H:%M',
    skipfooter=2,
    dtype={"Id": str, "Contact Name": str}
)

deals.name = 'deals'
display(deals.head(3))
mvf.my_df_info(deals,'deals')

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,...,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch
0,5805028000056864695,Ben Hall,NaN,NaN,New Lead,NaN,/eng/test,03.07.23women,NaN,v16,...,NaN,NaN,2024-06-21 15:30:00,NaN,NaN,NaN,NaN,5805028000056849495,NaN,NaN
1,5805028000056859489,Ulysses Adams,NaN,NaN,New Lead,NaN,/at-eng,NaN,NaN,NaN,...,Web Developer,Morning,2024-06-21 15:23:00,6.0,NaN,0,2000,5805028000056834471,NaN,NaN
2,5805028000056832357,Ulysses Adams,21.06.2024,D - Non Target,Lost,Non target,/at-eng,engwien_AT,00:26:43,b1-at,...,NaN,NaN,2024-06-21 14:45:00,NaN,NaN,NaN,NaN,5805028000056854421,NaN,NaN


'deals' DataFrame size (row, column): (21593, 23)
DataTypes info:
Id                             object
Deal Owner Name                object
Closing Date                   object
Quality                        object
Stage                          object
Lost Reason                    object
Page                           object
Campaign                       object
SLA                            object
Content                        object
Term                           object
Source                         object
Payment Type                   object
Product                        object
Education Type                 object
Created Time           datetime64[ns]
Course duration               float64
Months of study               float64
Initial Amount Paid            object
Offer Total Amount             object
Contact Name                   object
City                           object
Level of Deutsch               object
dtype: object
Number of missing values:
Id                  

### 1.4.2 Предварительная очистка и обработка:

In [28]:
# DataFrame pre-cleaning
mvf.df_pre_clean(deals)

DataFrame 'deals' info:

ROWs number: 21593
COLUMNs number: 23
No ROWs, where all values are NaN found
No duplicated ROWs found.
No COLUMNs with all NaN values were found.


Old columns names: ['Id', 'Deal Owner Name', 'Closing Date', 'Quality', 'Stage', 'Lost Reason', 'Page', 'Campaign', 'SLA', 'Content', 'Term', 'Source', 'Payment Type', 'Product', 'Education Type', 'Created Time', 'Course duration', 'Months of study', 'Initial Amount Paid', 'Offer Total Amount', 'Contact Name', 'City', 'Level of Deutsch']

New columns names: ['Id', 'Deal_Owner_Name', 'Closing_Date', 'Quality', 'Stage', 'Lost_Reason', 'Page', 'Campaign', 'SLA', 'Content', 'Term', 'Source', 'Payment_Type', 'Product', 'Education_Type', 'Created_Time', 'Course_duration', 'Months_of_study', 'Initial_Amount_Paid', 'Offer_Total_Amount', 'Contact_Name', 'City', 'Level_of_Deutsch']



#### Вывод по предварительной очистке датафрейма Deals:
- пустые колонки отсутствуют;
- пустые строки отсутствуют
- дублирующихся строк нет;

### 1.4.3 Обработка отсутствующих значений и преобразование типов данных   

#### Анализ данных по колонкам

In [29]:
# Analyzing columns 
mvf.my_columns_describe(deals)


DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21593 entries, 0 to 21592
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21593 non-null  object        
 1   Deal_Owner_Name      21564 non-null  object        
 2   Closing_Date         14645 non-null  object        
 3   Quality              19340 non-null  object        
 4   Stage                21593 non-null  object        
 5   Lost_Reason          16124 non-null  object        
 6   Page                 21593 non-null  object        
 7   Campaign             16067 non-null  object        
 8   SLA                  15533 non-null  object        
 9   Content              14147 non-null  object        
 10  Term                 12454 non-null  object        
 11  Source               21593 non-null  object        
 12  Payment_Type         496 non-null    object        
 13  Product       

####  Предложения по преобразованию типов в датафрейме DEALS:

In [30]:
# '''
# Id                      object -> object, так как это идентификатор
# Deal_Owner_Name         object -> category - имена повторяются и их набор фиксирован *
# Closing_Date            object -> DataTime - для анализа временных рядов 
# Quality                 object -> category - имена повторяются и их набор фиксирован *
# Stage                   object -> category - имена повторяются и их набор фиксирован *
# Lost_Reason             object -> category - имена повторяются и их набор фиксирован *
# Page                    object -> category - имена повторяются и их набор фиксирован *
# Campaign                object -> category - имена повторяются и их набор фиксирован *
# SLA                     object -> Timedelta   - для дальнейшего анализа (исправляем ошибку гомогенности)
# Content                 object -> category - имена повторяются и их набор фиксирован *
# Term                    object -> category - имена повторяются и их набор фиксирован *
# Source                  object -> category - имена повторяются и их набор фиксирован *
# Payment_Type            object -> category - имена повторяются и их набор фиксирован *
# Product                 object -> category - имена повторяются и их набор фиксирован *
# Education_Type          object -> category - имена повторяются и их набор фиксирован *
# Created_Time          DataTime -> DataTime    - оставить без изменения 
# Course_duration        float64 -> Int8 целое число
# Months_of_study        float64 -> Int8 целое число
# Initial_Amount_Paid    float64 -> float64 для корректного преобразования и поддержки NaN (исправляем ошибку гомогенности)
# Offer_Total_Amount     float64 -> float64 для корректного преобразования и поддержки NaN (исправляем ошибку гомогенности)
# Contact_Name           object  -> object, так как это идентификатор
# City                    object -> category - имена повторяются и их набор фиксирован *
# Level_of_Deutsch        object-> category - имена повторяются и их набор фиксирован 
#                                   (исправляем ошибку гомогенности)
# * ускорит операции фильтрации и группировки
# '''

#### Initial_Amount_Paid и Offer_Total_Amount

In [31]:
print("Initial_Amount_Paid unique before cleaning:\n",deals['Initial_Amount_Paid'].unique())
print("Offer_Total_Amount unique before cleaning:\n",deals['Offer_Total_Amount'].unique())

Initial_Amount_Paid unique before cleaning:
 [nan 0 1000 '€ 3.500,00' 500 100 4500 300 200 2000 11000 4000 3000 3500
 11500 1200 1500 1 5000 600 700 350 9 400 450]
Offer_Total_Amount unique before cleaning:
 [nan 2000 9000 11000 3500 4500 '€ 2.900,00' 6500 4000 3000 10000 2500 5000
 11500 1 1000 1200 0 1500 '€ 11398,00' 11111 6000]


In [32]:
# function for cleaning and converting data
def clean_and_convert_currency(column):
    # Clean the string by removing currency symbols, spaces, and unnecessary characters
    column = column.astype(str).str.replace('€', '', regex=False).str.replace(' ', '', regex=False)
    column = column.str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    # Convert the cleaned strings into numeric format
    return pd.to_numeric(column, errors='coerce')

# Apply the function to the required columns
deals['Initial_Amount_Paid'] = clean_and_convert_currency(deals['Initial_Amount_Paid'])
deals['Offer_Total_Amount'] = clean_and_convert_currency(deals['Offer_Total_Amount'])

# Output unique values for verification
print(Fore.GREEN + f"\nInitial_Amount_Paid unique after cleaning:\n{deals['Initial_Amount_Paid'].unique()}")
print(f"\nOffer_Total_Amount unique after cleaning:\n{deals['Offer_Total_Amount'].unique()}"+Style.RESET_ALL)

# Display the cleaned columns for verification
#display(deals[['Initial_Amount_Paid', 'Offer_Total_Amount']])


Initial_Amount_Paid unique after cleaning:
[     nan 0.00e+00 1.00e+03 3.50e+03 5.00e+02 1.00e+02 4.50e+03 3.00e+02
 2.00e+02 2.00e+03 1.10e+04 4.00e+03 3.00e+03 1.15e+04 1.20e+03 1.50e+03
 1.00e+00 5.00e+03 6.00e+02 7.00e+02 3.50e+02 9.00e+00 4.00e+02 4.50e+02]

Offer_Total_Amount unique after cleaning:
[       nan 2.0000e+03 9.0000e+03 1.1000e+04 3.5000e+03 4.5000e+03
 2.9000e+03 6.5000e+03 4.0000e+03 3.0000e+03 1.0000e+04 2.5000e+03
 5.0000e+03 1.1500e+04 1.0000e+00 1.0000e+03 1.2000e+03 0.0000e+00
 1.5000e+03 1.1398e+04 1.1111e+04 6.0000e+03]


In [33]:
# Checking the correctness of the entered data Initial_Amount_Paid must be <= Offer_Total_Amount
# if true, then align
non_correct_Initial_Amount_Paid = len(deals[deals['Initial_Amount_Paid'] > deals['Offer_Total_Amount']])
if  non_correct_Initial_Amount_Paid > 0:
    deals.loc[deals['Initial_Amount_Paid'] > deals['Offer_Total_Amount'], 'Initial_Amount_Paid'] = deals['Offer_Total_Amount']
    print(Fore.RED + f"Replaced {non_correct_Initial_Amount_Paid} incorrect 'Initial_Amount_Paid' values"+Style.RESET_ALL)

Replaced 58 incorrect 'Initial_Amount_Paid' values


#### SLA 
- <class 'datetime.time'>         13672
- <class 'datetime.timedelta'>     1861
- создаем новый столбец SLA_sec (приводим все значения в секунды) и затем восстанавливем оригинальный SLA с типом Timedelta
- отсутствующие значения оставляем NaN

In [34]:
from datetime import time, timedelta

# Define a function for converting time to seconds
def time_str_to_seconds(time_str):
    if pd.isna(time_str):  # Check if the value is NaN
        return 0
    if isinstance(time_str, (int, float)):  # If it's already a number, return it as is
        return int(time_str)
    if isinstance(time_str, time):  # Handle objects of type 'time'
        return time_str.hour * 3600 + time_str.minute * 60 + time_str.second
    if isinstance(time_str, timedelta):  # Handle objects of type 'timedelta'
        return int(time_str.total_seconds())
    try:
        # If it's a string in the format 'HH:MM:SS', convert it
        hours, minutes, seconds = map(int, time_str.split(':'))
        return hours * 3600 + minutes * 60 + seconds
    except ValueError:
        # If the format is not recognized, return 0
        return 0

# Apply the function to all rows in the 'SLA' column
deals['SLA_sec'] = deals['SLA'].apply(time_str_to_seconds)

deals['SLA'] = deals['SLA_sec'].apply(lambda x: timedelta(seconds=x))
display(deals[['SLA', 'SLA_sec']].sort_values(by='SLA',ascending=False))

,SLA,SLA_sec
21456,311 days 10:34:24,26908464
20045,227 days 16:54:57,19673697
18971,217 days 07:51:48,18777108
17653,207 days 16:31:32,17944292
18611,207 days 04:20:30,17900430
...,...,...
2903,0 days 00:00:00,0
13238,0 days 00:00:00,0
2904,0 days 00:00:00,0
2906,0 days 00:00:00,0


In [35]:
# converting data types

# Deal_ Owner_Name: object -> category
deals.Deal_Owner_Name = deals.Deal_Owner_Name.astype('category')

# Closing_Date: object -> datetime (left NaN)
deals.Closing_Date = pd.to_datetime(deals.Closing_Date, format='%d.%m.%Y', dayfirst=True, errors='coerce')

# Quality: object -> category
deals.Quality = deals.Quality.astype('category')

# Stage: object -> category
deals.Stage = deals.Stage.astype('category')

# Lost_Reason: object -> category
deals.Lost_Reason = deals.Lost_Reason.astype('category')

# Page: object -> category
deals.Page = deals.Page.astype('category')

# Campaign: object -> category
deals.Campaign = deals.Campaign.astype('category')

# Content: object -> category
deals.Content = deals.Content.astype('category')

# Term: object -> category
deals.Term = deals.Term.astype('category')

# Source: object -> category
deals.Source = deals.Source.astype('category')

# Payment_Type: object -> category
deals.Payment_Type = deals.Payment_Type.astype('category')

# Product: object -> category
deals.Product = deals.Product.astype('category')

# Education_Type: object -> category
deals.Education_Type = deals.Education_Type.astype('category')

# Course_duration: float64 -> Int8
deals.Course_duration = deals.Course_duration.astype('Int8')

# Months_of_study: float64 -> Int8
deals.Months_of_study = deals.Months_of_study.astype('Int8')

# Level_of_Deutsch: object -> category
deals.Level_of_Deutsch = deals.Level_of_Deutsch.astype('category')

In [36]:
mvf.my_df_info(deals,'deals')

'deals' DataFrame size (row, column): (21593, 24)
DataTypes info:
Id                              object
Deal_Owner_Name               category
Closing_Date            datetime64[ns]
Quality                       category
Stage                         category
Lost_Reason                   category
Page                          category
Campaign                      category
SLA                    timedelta64[ns]
Content                       category
Term                          category
Source                        category
Payment_Type                  category
Product                       category
Education_Type                category
Created_Time            datetime64[ns]
Course_duration                   Int8
Months_of_study                   Int8
Initial_Amount_Paid            float64
Offer_Total_Amount             float64
Contact_Name                    object
City                            object
Level_of_Deutsch              category
SLA_sec                          int6

#### Предложения по замене пропущенных значений в датафрейме DEALS:

- **Deal_Owner_Name** - пропущенные значения заполним значениями из справочника contacts, если есть совпадения.
- **Closing_Date** - для обеспечения целостности данных использовали при преобразовании типов ***errors ='coerce'***, чтобы оставить пропуски 'NaT', т.к. пропущенные знанения могут означать, что сделка еще не закрыта или дата закрытия не была зарегистрирована.
- **Quality** - пропущенные значения заполним значением "unknown".
- **Stage** -  **нет пропущенных значений**.
- **Lost Reason** - пропущенные значения заполним значением "unknown", значения "Duplicate" в колонке игнорируем, ввиду некорректного заполнения данных мернеджерами и отсутствием обратной связи из за специфики проекта
- **Page** -  **нет пропущенных значений**..
- **Campaign** - пропущенные значения заполним значением "unknown".
- **SLA** - пропущенные значения заполнили нулями при корректном расчете временных интервалов.
- **Content** - пропущенные значения заполним значением "unknown".
- **Term** - пропущенные значения заполним значением "unknown" ***предварительно заполнив где возможно данными взятыми в Campaign***.
- **Source** - **нет пропущенных значений**.
- **Payment_Type** - пропущенные значения заполним значением "unknown".
- **Product** - пропущенные значения заполним значением "unknown", ***предварительно заполнив*** где возможно, данными полученными на основе анализа Campaign и  Term***.
- **Education_Type** - пропущенные значения заполним случайным значением Morning/Evening, однако **пропорционально** исходному соотношению датафрейма, предварительно сгруппировав по Product. 
- **Course_duration** - предполагалось заменить пропущенные значения ***значением моды*** взятым из группы по Product и Education Type. Однако при замене выяснилось, что из-за слишком большого количества пропущенных значений результат только ухудшился, поэтому решение - **оставить колонку как есть**.
- **Months_of_study** - пропущенные значения оставляем, чтобы не нарушить целостность данных.
- **Initial_Amount_Paid** - пропущенные значения оставляем, чтобы не нарушить целостность данных. Используем значение как признак  стадии сделки (начало обсуждения с лидом условий контракта) на этапе до Stage - "Payment Done";  
- **Offer_Total_Amount** - пропущенные значения ***заполним медианным значением*** из группировки по Product, Education Type и Course_duration.
- **Contact_Name** - пропущенные значения оставляем, чтобы не нарушить целостность данных. Так как это идентификатор, то при анализе для корректного объединения с другими датафреймами по этому полю будем использовать соотвествующий тип Join.
- **City** - проверим введенные значения на **ошибки ввода**, используем модуль **geopy** для определения стран и координат, пропущенные значения заполним "unknown".
- **Level of Deutsch** - проведем **лексической обработку** и заполним некорректный ввод **таргетными значениями**, пропущенные значения - заполним значением "unknown".
- Для корректрого расчета юнит-экономики, а также значений суммы платежей и среднего платежа добавим в датафрейм три колонки:
    - **Paid** (оплаченная клиентом сумма);
    - **Т** (число транзакций);
    - **AOV** (средний чек)
    - **Логика их расчета**:
    - **если** **Months_of_study == 0**, то Paid = 0, T = 0, AOV = 0 - клиент еще не получил уcлугу и может изменить решение об обучении, соотвественно это платеж НЕ УЧИТЫВАЕТСЯ;
    - **иначе**:
        - ***если*** **Initial_Amount_Paid == Offer_Total_Amount**, то
            - Paid = (Offer_Total_Amount /Course_duration) * Months_of_study;
            - T = Months_of_study;
            - AOV = Paid / T;
        - ***иначе***:
            - Paid = Initial_Amount_Paid + (Offer_Total_Amount -Initial_Amount_Paid) / (Course_duration"] - 1) * Months_of_study
            - T = Months_of_study;
            - AOV = Paid / T.
**P.S.:**
- "unknown" выбрано для категориальных столбцов, чтобы устранить пропуски и обеспечить целостность данных.
- заполнение числовых столбцов медианным значением проведем с целью нивелирования смещения данных

#### Обработка отсутствующих значений DEALS на базе вышеуказанной логики

#### Deal_Owner_Name

In [37]:
# Загружаем контакты
contacts = pd.read_pickle("02_contacts.pkl")

print(Fore.RED + f"Deal_Owner_Name missing values before processing: {deals['Deal_Owner_Name'].isna().sum()}" + Style.RESET_ALL)

# Приводим `Deal_Owner_Name` к строковому типу
deals["Deal_Owner_Name"] = deals["Deal_Owner_Name"].astype("str")

# Создаем словарь для замены
contact_map = contacts.set_index("Id")["Contact_Owner_Name"].to_dict()

# Проверяем, есть ли совпадения
matching_ids = deals["Contact_Name"].isin(contact_map.keys())

if matching_ids.any():
    # Заполняем `NaN`, если есть совпадение в словаре
    deals.loc[deals["Deal_Owner_Name"]=="nan", "Deal_Owner_Name"] = deals["Contact_Name"].map(contact_map)
else:
    print(Fore.RED + "⚠ No matching Contact_Name found in contacts. No replacements made." + Style.RESET_ALL)

# Приводим обратно `"nan"` (строку) к `NaN`, если он случайно появился
deals["Deal_Owner_Name"] = deals["Deal_Owner_Name"].replace("nan", np.nan)

# Возвращаем категориальный тип данных
deals["Deal_Owner_Name"] = deals["Deal_Owner_Name"].astype("category")

# Проверяем оставшиеся `NaN`
print(Fore.BLUE + f"Deal_Owner_Name missing values after processing: {deals['Deal_Owner_Name'].isna().sum()}" + Style.RESET_ALL)


Deal_Owner_Name missing values before processing: 29
Deal_Owner_Name missing values after processing: 1


In [38]:
#Counter(list(deals.Deal_Owner_Name.unique()))

#### Education_Type

In [39]:
print(Fore.RED + f"Education_Type missing values before processing: {deals['Education_Type'].isna().sum()}"+ Style.RESET_ALL)

# Рассчитываем пропорцию Morning/Evening в имеющихся данных
education_counts = deals["Education_Type"].value_counts(normalize=True)
morning_ratio = education_counts.get("Morning", 0.5)  # Если нет данных, берем 50%
evening_ratio = education_counts.get("Evening", 0.5)  # Если нет данных, берем 50%

# Создаем копию DataFrame, чтобы избежать SettingWithCopyWarning
deals = deals.copy()

# Фильтруем только те строки, где Education_Type = NaN
mask_nan = deals["Education_Type"].isna()

# Группируем данные по Product и Course_duration и применяем замену
for product, group in deals[mask_nan].groupby("Product", observed=True):
    num_missing = group.shape[0]                    # Количество отсутствующих значений
    num_morning = int(num_missing * morning_ratio)  # Сколько Morning заполняем
    num_evening = num_missing - num_morning         # Остальное Evening

    # Генерируем случайные значения для замены, но пропорционально
    new_values = np.array(["Morning"] * num_morning + ["Evening"] * num_evening)
    np.random.shuffle(new_values)  # Перемешиваем значения
    
    # Заполняем в оригинальном DataFrame
    deals.loc[group.index, "Education_Type"] = new_values


# Check if there are any missing values
print(Fore.BLUE + f"Education_Type missing values after processing: {deals['Education_Type'].isna().sum()}"+ Style.RESET_ALL)

Education_Type missing values before processing: 18294
Education_Type missing values after processing: 17992


#### Course_duration

In [40]:
from statistics import mode
import numpy as np

print(Fore.RED + f"Course_duration missing values before processing: {deals['Course_duration'].isna().sum()}"+ Style.RESET_ALL)

# Check that the 'Course_duration' column contains at least one value before mode()
deals.loc[deals.Stage == 'Payment Done', 'Course_duration'] = (
    deals[deals.Stage == 'Payment Done']
    .groupby(['Product', 'Education_Type'], observed=False)['Course_duration']    
    .transform(lambda x: x.fillna(mode(x.dropna()) if not x.dropna().empty else np.nan))
)

# Check if there are any missing values
print(Fore.BLUE + f"Course_duration missing values after processing: {deals['Course_duration'].isna().sum()}"+ Style.RESET_ALL)

#pd.set_option("display.max_rows", 20)

#deals[['Product', 'Education_Type', 'Course_duration']].dropna()


Course_duration missing values before processing: 18006
Course_duration missing values after processing: 18006


In [41]:
# нет результата из за большого количества NaN -> Course_duration лучше оставить как есть

#### Term

In [42]:
# Function to fill missing Term(=AdGroup) values based on Campaign (values analyzed using LMM)
# Input data: unique values lists of Campaign and Term columns

def fill_adgroup(row: pd.Series) -> str:
    """
    Fills missing 'Term' values based on 'Campaign' column using predefined logic.

    Parameters:
    row (pd.Series): A row of the DataFrame containing 'Campaign' and 'Term' columns.

    Returns:
    str: The appropriate 'Term' value based on keywords in 'Campaign', or the existing value if 'Term' is not missing.

    Details:
    - Checks if the 'Term' value is missing (NaN).
    - Converts the 'Campaign' value to lowercase for case-insensitive processing.
    - Searches for specific keywords in 'Campaign' and assigns corresponding 'Term' values:
        - Keywords like 'wide', 'recentlymoved', 'women', etc., are mapped to respective 'Term' categories.
        - Additional nested logic is applied for keywords such as 'com' or 'interests'.
    - If no keywords match, assigns a default value ('b').
    """
    if pd.isnull(row['Term']) and isinstance(row['Campaign'], str):  # Check if Term is missing and Campaign is a string
        campaign = row['Campaign'].lower()  # Convert to lowercase for consistent matching
        # Mapping Term values based on keywords
        if 'wide' in campaign:
            return 'wide'
        elif 'recentlymoved' in campaign:
            return 'recentlymoved'
        elif 'women' in campaign:
            return 'women'
        elif 'interests' in campaign:
            if 'webdev' in campaign:
                return 'interest_work_WebDev'
            elif 'programming' in campaign:
                return 'interest_programming_WebDev'
            elif 'dataanalytics' in campaign:
                return 'interest_dataanalytics'
            else:
                return 'interest_all'
        elif 'lal' in campaign:
            return 'LAL1'
        elif 'retargeting' in campaign:
            return 'retargeting'
        elif 'discovery' in campaign:
            return 'discovery'
        elif 'python-developer' in campaign:
            return 'interest_python-developer'
        elif 'qa-engineer' in campaign:
            return 'wide_qa-engineer'
        elif 'webdesigner' in campaign:
            return 'wide_webdesigner'
        elif 'accountant' in campaign:
            return 'accountant_wide'
        elif 'berlin' in campaign:
            return 'berlin_wide'
        elif 'com' in campaign:
            if 'august' in campaign:
                return 'Com_august'
            elif 'march' in campaign:
                return 'Com_march'
            elif 'july' in campaign:
                return 'Com_july_1'
        elif 'promoposts' in campaign:
            return 'promoposts_b'
        else:
            return 'b'  # Default value for unmatched cases
    return row['Term']  # Return existing value if Term is not missing


# Tracking the number of missing values before and after processing
# Number of missing values in 'Term' before processing
num_NaN_before = deals['Term'].isna().sum()

# Applying 'fill_adgroup' to fill missing Term values
deals['Term'] = deals.apply(fill_adgroup, axis=1)

# Number of missing values in 'Term' after processing
num_NaN_after = deals['Term'].isna().sum()

# Calculate the number of processed rows and the percentage of missing values processed
processed_rows = num_NaN_before - num_NaN_after
percent_processed = (processed_rows / num_NaN_before) * 100  # Percentage of processed NaN values

# Output the results
print(Fore.GREEN + f"Number of NaN records in 'Term' before processing = {num_NaN_before}")
print(f"Number of NaN records in 'Term' after processing = {num_NaN_after}")
print(f"Processed NaN are {processed_rows} values, which is {percent_processed:.2f}%." + Style.RESET_ALL)

Number of NaN records in 'Term' before processing = 9139
Number of NaN records in 'Term' after processing = 4686
Processed NaN are 4453 values, which is 48.73%.


In [43]:
# do a search by keywords and determine the advertising campaign
deals[deals['Campaign'].str.contains('brand_search_eng_DE', case=False, na=False)][['Campaign', 'Term', 'Content']].head()

,Campaign,Term,Content
7,brand_search_eng_DE,it career hub,152789402780_{region_name}_695563281558
25,brand_search_eng_DE,it career hub,152789402780_{region_name}_695563281558
75,brand_search_eng_DE,it career hub,152789402780_{region_name}_668024583824
85,brand_search_eng_DE,it career hub,152789402780_{region_name}_695563281558
111,brand_search_eng_DE,ich,152789402780_{region_name}_695563281558


In [44]:
# replace the value of a subset in an advertising campaign
deals.loc[(deals['Campaign'] == 'brand_search_eng_DE'), 'Term'] = 'it career hub'

#### Product

**Анализируем подгруппу (Term) в рекламной кампании (Campaign) которая направлена на курс аналитик данных**

In [45]:
# Analyzing data analytics course advertising campaigns
pd.set_option("display.max_rows", 20)
deals.loc[deals['Campaign'] == 'gen_analyst_DE', ['Campaign','Term' ]]

,Campaign,Term
15639,gen_analyst_DE,аналитик данных обучение
15669,gen_analyst_DE,data analyst курсы
15693,gen_analyst_DE,курсы аналитик данных
15799,gen_analyst_DE,обучение на аналитика данных
15932,gen_analyst_DE,курсы аналитик данных
...,...,...
17863,gen_analyst_DE,data analysis analytics
17965,gen_analyst_DE,data analyst курсы
21522,gen_analyst_DE,data analyst
21532,gen_analyst_DE,курсы аналитика с нуля


**Фильтруем продукт лиды в который пришли по этой группе объявлений в рекламной кампании**

In [46]:
# Filter rows based on conditions r"дата|аналит|analys"
display(deals[
    #(deals.Stage.isin(['Payment Done'])) &
    (deals.Term.str.contains(r"дата|аналит|analys", case=False, na=False))
][['Product', 'Source', 'Campaign', 'Term']])

,Product,Source,Campaign,Term
15639,UX/UI Design,Google Ads,gen_analyst_DE,аналитик данных обучение
15669,NaN,Google Ads,gen_analyst_DE,data analyst курсы
15693,NaN,Google Ads,gen_analyst_DE,курсы аналитик данных
15799,NaN,Google Ads,gen_analyst_DE,обучение на аналитика данных
15932,NaN,Google Ads,gen_analyst_DE,курсы аналитик данных
...,...,...,...,...
17863,Digital Marketing,Google Ads,gen_analyst_DE,data analysis analytics
17965,NaN,Google Ads,gen_analyst_DE,data analyst курсы
21522,NaN,Google Ads,gen_analyst_DE,data analyst
21532,NaN,Google Ads,gen_analyst_DE,курсы аналитика с нуля


- Делаю предположение (которое при выполнении реального проекта обязательно согласовываю с отделом продаж),
что **лиды пришедшие по рекламе курсов датааналитики в атрибуте Product должны иметь пометку 'Data Analytics'** и соотвествующим образом **восстанавливаю пропущенные значения Product** и **заменяю ошибки ввода данных**

In [47]:
print("Product disribution:")
display(deals.groupby('Product',observed=True)['Product'].count())

# making an assumption (which I always agree with the sales department when implementing a real project); 
# that leads that come from advertising data analytics courses in the Product attribute should have the 'Data Analytics' mark;
# accordingly restore the missing Product values and replace data entry errors

# replace the value of a subset in an advertising campaign
#deals.loc[deals['Campaign'] == 'gen_analyst_DE', ['Term', 'Product']] = ['data analyst', 'Data Analytics']
deals.loc[deals['Campaign'] == 'gen_analyst_DE', 'Product'] = 'Data Analytics'

# Check replacing 
display(deals[
    #(deals.Stage.isin(['Payment Done'])) &
    (deals.Term.str.contains(r"дата|аналит|analys", case=False, na=False))
][['Product', 'Source', 'Campaign', 'Term']])
print("Final Product disribution:")
display(deals.groupby('Product',observed=True)['Product'].count())

Product disribution:


Product
Data Analytics            1
Digital Marketing      1990
Find yourself in IT       4
UX/UI Design           1022
Web Developer           575
Name: Product, dtype: int64

,Product,Source,Campaign,Term
15639,Data Analytics,Google Ads,gen_analyst_DE,аналитик данных обучение
15669,Data Analytics,Google Ads,gen_analyst_DE,data analyst курсы
15693,Data Analytics,Google Ads,gen_analyst_DE,курсы аналитик данных
15799,Data Analytics,Google Ads,gen_analyst_DE,обучение на аналитика данных
15932,Data Analytics,Google Ads,gen_analyst_DE,курсы аналитик данных
...,...,...,...,...
17863,Data Analytics,Google Ads,gen_analyst_DE,data analysis analytics
17965,Data Analytics,Google Ads,gen_analyst_DE,data analyst курсы
21522,Data Analytics,Google Ads,gen_analyst_DE,data analyst
21532,Data Analytics,Google Ads,gen_analyst_DE,курсы аналитика с нуля


Final Product disribution:


Product
Data Analytics           36
Digital Marketing      1978
Find yourself in IT       4
UX/UI Design           1019
Web Developer           575
Name: Product, dtype: int64

#### Cleaning Level_of_Deutsch

In [48]:
deals.Level_of_Deutsch.unique()

[NaN, 'в1', 'A2', 'б1', 'b1', ..., 'точно уровень не знаю, но говорить могу - учи..., 'А2-В1 учит', 'В1 (учится на В2 уже)', 'В январе - В2 сдает', 'b1 должна получить результаты в феврале']
Length: 216
Categories (215, object): [0, 90, '-', '.', ..., 'с1', 'сдавала А2 в сентябре', 'точно уровень не знаю, но говорить могу - учи..., 'учит A2']

In [49]:
# Function to clean Level_of_Deutsch
import re 
def clean_value(value):
    if value == 'unknown':
        return 'unknown'

    value = str(value).strip().lower()

    # Use regular expressions to match specific keywords for each proficiency level
    if re.search(r'\b(a0|а0|0|нет|нулевой|никакой|не знаю|не учил|не учила|[-])\b', value):
        return 'A0'
    elif re.search(r'\b(a1|а1|a|а|живет|разговор|говорить могу)\b', value):
        return 'A1'
    elif re.search(r'\b(a2|f2|а2)\b', value):
        return 'A2'
    elif re.search(r'\b(b1|б1|в1|b|б|бй|б10б2|гражданка|гражданин)\b', value):
        return 'B1'
    elif re.search(r'\b(b2|б2|в2)\b', value):
        return 'B2'
    elif re.search(r'\b(c1|с1|с)\b', value):
        return 'C1'
    elif re.search(r'\b(c2|с2)\b', value):
        return 'C2'
    else:
        return 'unknown'  # If no match is found, return 'unknown'


if 'unknown' not in deals['Level_of_Deutsch'].cat.categories:
    deals['Level_of_Deutsch'] = deals['Level_of_Deutsch'].cat.add_categories(['unknown'])
deals['Level_of_Deutsch'] = deals['Level_of_Deutsch'].fillna('unknown')

# Applying clean_value() to fill missing Level_of_Deutsch values
deals.Level_of_Deutsch = deals['Level_of_Deutsch'].apply(clean_value)

deals.Level_of_Deutsch = deals.Level_of_Deutsch.astype('category')

# Count occurrences of unique values after cleaning
result = Counter(deals['Level_of_Deutsch'].tolist())
print(result)

Counter({'unknown': 20375, 'B1': 780, 'B2': 162, 'A2': 137, 'A0': 85, 'A1': 26, 'C1': 25, 'C2': 3})


#### Cleaning City

**Для дальнейшего анализа** нам потребуется собственно название города (City), а также страна (Country) и географические координаты (Latitude,Longitude).
- для верификации города и определения дополнительных полей создадим функцию **check_address()** на основе модуля **geopy** 

**Шаги обработки**:
- на самом деле в поле City содержится микс от собственно города, до полного адреса с использованием кирилицы и уникальных символов немецкого, румынского, молдавского и алфавитов и других славянских языков;
- выделим уникальные значения City в список city_list;
- напишем функцию clean_addresses(), которыя определит некорректно заполненные поля City (с учетом вышеназванных проблем заполнения поля) и запишет их в список wrong_addresses;
- на основе wrong_addresses создадим словарь некорректных значений и исправим данные в датафрейме, заполнив отсутствующие значения и "мусор" значением "unknown";
- создадим собственно функцию check_address() и проверифицируем город, а также найдем необходимые поля Country,Latitude,Longitude.

#### ВНИМАНИЕ! Процесс обработки данных функцией **check_address()** может занять более 30 минут в зависимости от характеристик компьютера и скорости интернета. Прогресс бар - показывает процент выполнения процесса. 

In [50]:
# list of unique City values
city_list = deals.City.unique().tolist()
print(f"Count of City unique: {len(city_list)}")
print("First 100 values:")
print(city_list[:100],"...")

Count of City unique: 877
First 100 values:
[nan, 'Crailsheim', 'Dortmund', 'Stuttgart', 'München', 'Berlin', 'Wien', 'Offenbach am Main', 'Eberbach', 'Görlitz', 'Pfedelbach', 'Unterhaching', 'Karl-Liebknecht str. 24, Hildburghausen, Thüringen', 'Rüdesheim am Rhein', 'Dresden', 'Gummersbach', '-', 'Kassel', 'Wenzenbach', 'Merseburg', 'Gommern', 'Pommelsbrunn', 'Duisburg', 'Herzogenrath', 'Schwandorf', 'Mainz', 'Podskalie', 'Zinnowitz', 'Quedlinburg', 'Poland , Gdansk , Al. Grunwaldzka 7, ap. 1a', 'Wolfsburg', 'Weilburg', 'Dillenburg', 'Neu-Ulm', 'Lauter-Bernsbach', 'Bonn', 'Riedstadt', 'Rosenheim', 'Mönchengladbach', 'Neuburg', 'Rostock', 'Bad Oeynhausen', 'Chemnitz', 'Diez', 'Nürnberg', 'Laubach', 'Düren', 'Düsseldorf', 'Zwickau', 'Bremen', 'Halle', 'Erbach', 'Jünkerath', 'Magdeburg', 'Celle', 'Germering', 'Kleve', 'Leinfelden-Echterdingen', 'Garmisch-Partenkirchen', 'Leipzig', 'Hof', 'Lünen', 'Murr', 'Bochum', 'Leonberg', 'Bad Homburg vor der Höhe', 'Kiel', 'Theres', 'Lüchow', 'Stolb

In [51]:
def clean_addresses(addresses):
    clean_addr = []
    wrong_addr = []
    for address in addresses:
        # Check if the address matches the pattern
        # Added umlaut characters äöüÄÖÜß: and ß, which are used in German
        # Unique characters of the Polish alphabet: ąćęłńóśźż
        # Unique characters of the Moldovan and Romanian alphabets: ĂÂÎȘȚăâîșț
        # Unique characters of the Ukrainian alphabet: ҐЄІЇЎґєіїў
        # Added characters ( and ) for working with strings containing text in parentheses
        # Added forward slash and hyphen to the set of allowed characters / , -:
        
        pattern = r'^[A-Za-zА-Яа-яЁёäöüÄÖÜßąćęłńóŚśźżĂÂÎȘȚăâîșțҐЄІЇЎґєіїўІіЇїґєІіЇї\s\‑\-/]+$'
        if address is np.nan or address == '-' or not re.match(pattern, address) or ',' in address:
            wrong_addr.append(address)
        else:
            clean_addr.append(address)
    return clean_addr, wrong_addr


print(len(city_list))
cleaned_addresses, wrong_addresses = clean_addresses(city_list)
print(len(cleaned_addresses))
print(len(wrong_addresses))
print(wrong_addresses)

877
868
9
[nan, 'Karl-Liebknecht str. 24, Hildburghausen, Thüringen', '-', 'Poland , Gdansk , Al. Grunwaldzka 7, ap. 1a', 'Vor Ebersbach 1, 77761 Schiltach', 'Nebra (Unstrut)', 'Halle (Saale)', 'Eisenbach (Hochschwarzwald)', 'Lindau (Bodensee)']


In [52]:
# Fill missing "City" values with "unknown"
# if 'unknown' not in deals["City"].cat.categories:
#         deals["City"] = deals["City"].cat.add_categories(['unknown'])    
deals["City"] = deals["City"].fillna("unknown")

# Replace incorrect addresses (input errors) with city names
deals["City"] = deals["City"].replace({
    "Karl-Liebknecht str. 24, Hildburghausen, Thüringen": "Hildburghausen",
    "Poland , Gdansk , Al. Grunwaldzka 7, ap. 1a": "Gdansk",
    "Vor Ebersbach 1, 77761 Schiltach": "Schiltach",
    "-": "unknown"
})
deals.loc[deals.City!="unknown","City"]

56       Crailsheim
60         Dortmund
65        Stuttgart
71          München
75           Berlin
            ...    
21313     Frankfurt
21314     Wolfsburg
21410    Ingolstadt
21551         Essen
21578         Brake
Name: City, Length: 2163, dtype: object

In [53]:
# list of unique City values
city_list = deals.City.unique().tolist()
print(f"Count of City unique: {len(city_list)}")
print(city_list[:100],"...")

Count of City unique: 876
['unknown', 'Crailsheim', 'Dortmund', 'Stuttgart', 'München', 'Berlin', 'Wien', 'Offenbach am Main', 'Eberbach', 'Görlitz', 'Pfedelbach', 'Unterhaching', 'Hildburghausen', 'Rüdesheim am Rhein', 'Dresden', 'Gummersbach', 'Kassel', 'Wenzenbach', 'Merseburg', 'Gommern', 'Pommelsbrunn', 'Duisburg', 'Herzogenrath', 'Schwandorf', 'Mainz', 'Podskalie', 'Zinnowitz', 'Quedlinburg', 'Gdansk', 'Wolfsburg', 'Weilburg', 'Dillenburg', 'Neu-Ulm', 'Lauter-Bernsbach', 'Bonn', 'Riedstadt', 'Rosenheim', 'Mönchengladbach', 'Neuburg', 'Rostock', 'Bad Oeynhausen', 'Chemnitz', 'Diez', 'Nürnberg', 'Laubach', 'Düren', 'Düsseldorf', 'Zwickau', 'Bremen', 'Halle', 'Erbach', 'Jünkerath', 'Magdeburg', 'Celle', 'Germering', 'Kleve', 'Leinfelden-Echterdingen', 'Garmisch-Partenkirchen', 'Leipzig', 'Hof', 'Lünen', 'Murr', 'Bochum', 'Leonberg', 'Bad Homburg vor der Höhe', 'Kiel', 'Theres', 'Lüchow', 'Stolberg', 'Bautzen', 'Weiden', 'Osterode am Harz', 'Schiltach', 'Göppingen', 'Ansbach', 'Aache

In [54]:
import time
from tqdm import tqdm  # For the progress bar
from geopy.geocoders import Nominatim

# Function to check address and return values: City,Country,Latitude,Longitude
def check_address(address):
    
    if address=='unknown':
        return pd.Series(['unknown', 'unknown', 'unknown', 'unknown'], index=['City', 'Country', 'Latitude', 'Longitude'])

    # Create a geolocator
    geolocator = Nominatim(user_agent="city_country_finder", timeout=5)
    try:
        location = geolocator.geocode(address, language='en', addressdetails=1)
        if location:
            city = location.address.split(',')[0].strip()
            country = location.address.split(',')[-1].strip()
            lat = location.latitude
            lon = location.longitude
            return pd.Series([city, country, lat, lon], index=['City', 'Country', 'Latitude', 'Longitude'])
        else:
            return pd.Series(['unknown', 'unknown', 'unknown', 'unknown'], index=['City', 'Country', 'Latitude', 'Longitude'])
    except Exception as e:
        print(f"Error processing address {address}: {e}")
        return pd.Series(['unknown', 'unknown', 'unknown', 'unknown'], index=['City', 'Country', 'Latitude', 'Longitude'])

tqdm.pandas()
# Applying clean_value() to fill missing Level_of_Deutsch values
deals[['City', 'Country', 'Latitude', 'Longitude']] = deals['City'].progress_apply(check_address)

# City: object -> category
deals.City = deals.City.astype('category')

pd.reset_option('display') 
deals.loc[deals.City != 'unknown', ['City', 'Country', 'Latitude', 'Longitude']]

100%|████████████████████████████████████████████████████████████████████████████| 21593/21593 [36:00<00:00,  9.99it/s]


,City,Country,Latitude,Longitude
56,Crailsheim,Germany,49.136563,10.072019
60,Dortmund,Germany,51.514227,7.465279
65,Stuttgart,Germany,48.778449,9.180013
71,Munich,Germany,48.137108,11.575382
75,Berlin,Germany,52.510885,13.398937
...,...,...,...,...
21313,Frankfurt,Germany,50.110644,8.682092
21314,Wolfsburg,Germany,52.420559,10.786168
21410,Ingolstadt,Germany,48.763016,11.42504
21551,Essen,Germany,51.458224,7.015817


#### Обработка аномалий в Created_Time > Closing_Date

In [55]:
# Counting anomalies (input errors) where 'Closing_Date' is less than 'Created_Time' and fixing

# Convert the Created_Time column to a date
deals['Created_Time_Date'] = deals['Created_Time'].dt.date

# Identify rows with anomalies
anomaly = deals['Closing_Date'] < deals['Created_Time_Date']
print(f"It's {anomaly.sum()} anomaly founded. Fixing it...")

# Save line numbers with anomalies
anomaly_indices = deals.index[anomaly]


# Handling rows with anomalies
for idx in anomaly_indices:
   # Replace date in 'Created_Time' with 'Closing_Date', keeping time
    closing_date = deals.at[idx, 'Closing_Date']
    deals.at[idx, 'Created_Time'] = pd.Timestamp(closing_date).replace(
        hour=deals.at[idx, 'Created_Time'].hour,
        minute=deals.at[idx, 'Created_Time'].minute,
        second=deals.at[idx, 'Created_Time'].second,
        microsecond=deals.at[idx, 'Created_Time'].microsecond
    )
    # Replacing 'Closing_Date' with the date from 'Created_Time_Date'
    created_date = deals.at[idx, 'Created_Time_Date']
    deals.at[idx, 'Closing_Date'] = pd.Timestamp(created_date)
# drop 'Created_Time_Date'
deals.drop(columns=['Created_Time_Date'], inplace=True)
print(deals[['Created_Time', 'Closing_Date']].loc[anomaly_indices])

It's 44 anomaly founded. Fixing it...
             Created_Time Closing_Date
454   2024-06-11 00:06:00   2024-06-16
2083  2024-05-22 21:29:00   2024-05-25
2787  2024-05-07 11:19:00   2024-05-12
3019  2024-05-07 15:31:00   2024-05-08
3022  2024-04-17 14:48:00   2024-05-08
3031  2024-05-07 12:54:00   2024-05-08
3691  2024-04-23 15:16:00   2024-04-30
4107  2024-04-17 17:30:00   2024-04-24
4169  2024-04-18 21:44:00   2024-04-23
4434  2024-04-18 08:57:00   2024-04-21
4520  2023-08-21 06:19:00   2024-04-20
4798  2024-04-12 09:10:00   2024-04-17
4906  2024-04-11 19:30:00   2024-04-16
5000  2024-04-10 15:35:00   2024-04-16
5088  2024-04-11 07:50:00   2024-04-16
5492  2024-04-08 04:14:00   2024-04-13
5657  2024-04-09 01:40:00   2024-04-12
6212  2024-04-03 15:02:00   2024-04-07
6357  2023-10-03 10:40:00   2024-04-05
7215  2024-03-19 16:44:00   2024-03-26
7622  2024-03-13 12:37:00   2024-03-20
7752  2024-03-12 10:56:00   2024-03-19
8655  2024-02-28 09:15:00   2024-03-06
8817  2024-02-29 08:17:00 

#### Рассчет по каждой сделке количества транзакций (Т), среднего чека (AOV) и оплаченной суммы (Paid)

In [56]:
# Apply the function fill_Paid from  My_Function_050824_M_Filimonov to the DataFrame
deals[["Paid", "T", "AOV"]] = deals.apply(mvf.fill_Paid, axis=1)

# check calculations
deals[(deals.Product !="unknown")& (deals.Education_Type !="unknown") & (deals.Stage=="Payment Done")]\
    [["Product","Education_Type","Payment_Type","Course_duration","Months_of_study","Initial_Amount_Paid","Offer_Total_Amount","Paid", "T", "AOV"]]

,Product,Education_Type,Payment_Type,Course_duration,Months_of_study,Initial_Amount_Paid,Offer_Total_Amount,Paid,T,AOV
468,Digital Marketing,Evening,Recurring Payments,11,1,500.0,4500.0,900.0,1.0,900.00
478,Digital Marketing,Evening,Recurring Payments,11,1,500.0,4500.0,900.0,1.0,900.00
497,Web Developer,Morning,NaN,6,1,1000.0,9000.0,2600.0,1.0,2600.00
833,Web Developer,Morning,NaN,6,1,1000.0,9000.0,2600.0,1.0,2600.00
1040,Digital Marketing,Morning,NaN,11,1,1000.0,11000.0,2000.0,1.0,2000.00
...,...,...,...,...,...,...,...,...,...,...
21314,Web Developer,Morning,NaN,6,2,1000.0,5000.0,2600.0,2.0,1300.00
21367,Digital Marketing,Morning,NaN,11,11,1000.0,11000.0,12000.0,11.0,1090.91
21410,Digital Marketing,Morning,One Payment,11,11,450.0,4000.0,4355.0,11.0,395.91
21555,Digital Marketing,Morning,NaN,11,8,1000.0,11500.0,9400.0,8.0,1175.00


#### Обработка оставшихся данных согласно принятому решению

In [57]:
# Fill in missing values for category data
category_columns = ['Quality', 'Stage', 'Lost_Reason', 'Page', 'Campaign', 'Content',
                      'Source', 'Payment_Type', 'Product','Education_Type']
for col in category_columns:
    if 'unknown' not in deals[col].cat.categories:
        deals[col] = deals[col].cat.add_categories(['unknown'])
    deals[col] = deals[col].fillna('unknown')

In [58]:
# Checking data after filling gaps and transformation
#deals.name = 'deals'
mvf.my_df_info(deals,'deals')

'deals' DataFrame size (row, column): (21593, 30)
DataTypes info:
Id                              object
Deal_Owner_Name               category
Closing_Date            datetime64[ns]
Quality                       category
Stage                         category
Lost_Reason                   category
Page                          category
Campaign                      category
SLA                    timedelta64[ns]
Content                       category
Term                            object
Source                        category
Payment_Type                  category
Product                       category
Education_Type                category
Created_Time            datetime64[ns]
Course_duration                   Int8
Months_of_study                   Int8
Initial_Amount_Paid            float64
Offer_Total_Amount             float64
Contact_Name                    object
City                          category
Level_of_Deutsch              category
SLA_sec                          int6

### 1.4.4 Общие выводы по датафрейму DEALS

- Датафрейм Deals содержит информацию по сделкам на обучение;
- До обработки датафрейм имел 21 593 строки и 23 столбца, объем 3.8 Мб
- После обработки он имеет 21593  строки , 30 столбцов (добавден столбец SLA_sec - время отклика в секундах и столбцы  Country - для анализа охвата регионов откуда студенты ,Latitude Longitude для точной идентификации геолокации, и 3 столбца для анализа юнит экономики),  объем 2.9 Мб - уменьшен за счет оптимизации типов данных;
- Проведение сопоставительного анализа в группах данных Campaign - Term позволило восстановить 48.73% отсутствующих значений в Term  и 35 значений в Product;
- Лексический анализ Level_of_Deutsch позволил сопоставить по категориям 1218 значений что составило 99% доступных данных в этом столбце;
- Обработка при помощи регуляного выражения позволила очистить 875 из 877 доступных значений данных City, дальнейшая обработка которых средствами библиотеки geopy позволило однозначно определить страну и геолокацию объектов, что позволит провести более качественно дальнейший анализ;
- Испарвление ошибок гомогенности данных в столбцах SLA, Initial_Amount_Paid, Offer_Total_Amount и Level_of_Deutsch, замена оставшихся NaN в категориальных столбцах на "unknown" позволило корректно заменить отсутствующие значения.

### 1.4.5 Сохранение DEALS 

In [59]:
# delete not informative columns
deals = deals.drop(columns=['SLA', 'Content',"Page"])

# write the resulting dataframe to an Excel file for further processing in Power BI
deals.to_excel('deals_cleaned.xlsx', index=False, engine='openpyxl')

#save DataFrame
deals.to_pickle("04_deals.pkl")

In [60]:
# Check all DataFrames 

In [63]:
# load DataFrames:
calls = pd.read_pickle("01_calls.pkl")
contacts = pd.read_pickle("02_contacts.pkl")
spend = pd.read_pickle("03_spend.pkl")
deals = pd.read_pickle("04_deals.pkl")

In [64]:
print("calls:")
print(calls.info())
print("contacts:")
print(contacts.info())
print("spend:")
print(spend.info())
print("deals:")
print(deals.info())

calls:
<class 'pandas.core.frame.DataFrame'>
Index: 95588 entries, 1 to 95873
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Call_Owner_Name       95588 non-null  category      
 1   Call_Type             95588 non-null  category      
 2   Id                    95588 non-null  object        
 3   Call_Start_Time       95588 non-null  datetime64[ns]
 4   CONTACTID             95588 non-null  object        
 5   Call_Duration         95588 non-null  int32         
 6   Call_Status           95588 non-null  category      
 7   Outgoing_Call_Status  95588 non-null  category      
 8   Scheduled_in_CRM      95588 non-null  category      
dtypes: category(5), datetime64[ns](1), int32(1), object(2)
memory usage: 3.7+ MB
None
contacts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 